In [5]:
#import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



In [66]:
# Define the ticker symbol for JPMorgan Chase & Co.
ticker_symbol = 'JPM'

# Initialize the yfinance Ticker object for JPM
jpm_ticker = yf.Ticker(ticker_symbol)

# Get available expiration dates
expiration_dates = jpm_ticker.options

# Choose an expiration date that exists in the list of available expiration dates
# For the purpose of this example, let's choose the first available expiration date
expiration_date = expiration_dates[0]

# Download the options data for JPM for the chosen expiration date
jpm_options = jpm_ticker.option_chain(expiration_date)

# Separate the calls and puts data
jpm_calls = jpm_options.calls
jpm_puts = jpm_options.puts

# Display the first few rows of the calls data

df = pd.DataFrame(jpm_calls)

In [68]:
df.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,JPM240126C00090000,2024-01-02 16:33:56+00:00,90.0,80.87,79.70,81.35,0.000000,0.000000,1.0,0,3.125002,True,REGULAR,USD
1,JPM240126C00100000,2024-01-24 18:50:36+00:00,100.0,70.45,69.85,71.00,0.089996,0.127908,1.0,5,3.939453,True,REGULAR,USD
2,JPM240126C00105000,2024-01-24 18:49:50+00:00,105.0,65.50,64.75,66.15,0.089996,0.137588,1.0,15,3.804688,True,REGULAR,USD
3,JPM240126C00130000,2024-01-03 15:23:05+00:00,130.0,40.80,39.70,40.90,0.000000,0.000000,NaN,0,2.121098,True,REGULAR,USD
4,JPM240126C00135000,2023-12-26 14:44:24+00:00,135.0,32.96,34.55,36.15,0.000000,0.000000,1.0,0,2.074224,True,REGULAR,USD


### Kraken API Request

In [74]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_historical_data(pair, interval, since):
    url = f'https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}&since={since}'
    response = requests.get(url).json()
    return pd.DataFrame(response['result'][pair], columns=['Time', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume', 'Count'])

def get_unix_timestamp(date):
    return int(datetime.timestamp(date))

# Define your pair and interval
pair = 'XXBTZUSD'  # Example for Bitcoin to USD
interval = 1440  # Daily data (1440 minutes in a day)

# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)  # 2 years back from today
since = get_unix_timestamp(start_date)

# Fetch the historical data
historical_data = fetch_historical_data(pair, interval, since)
historical_data['Close'] = historical_data['Close'].astype(float)

# Convert timestamp to readable date format for clarity
historical_data['Time'] = pd.to_datetime(historical_data['Time'], unit='s')


df = pd.DataFrame(historical_data)

In [76]:
df.head()

,Time,Open,High,Low,Close,VWAP,Volume,Count
0,2022-02-05,41597.2,41926.4,40883.7,41419.0,41509.6,2211.15688361,24916
1,2022-02-06,41419.0,42694.5,41141.9,42419.5,41772.3,1818.07180806,22192
2,2022-02-07,42426.1,44528.6,41697.4,43867.2,43382.3,4604.68183967,39555
3,2022-02-08,43873.1,45500.0,42709.6,44079.1,43881.5,3432.44974988,35663
4,2022-02-09,44083.1,44831.0,43187.2,44420.0,44013.4,3210.42939961,27148


### deribit API Request

In [ ]:
#MaW0sFMebW5vxx_WvodEHmgJ2bcqXLDXmMDgUsieesQ
#Hrykz9ld



In [4]:
""" Download CC-Option Data from Deribit via public API """

"""
Matteo Bottacini -- matteo.bottacini@usi.ch
"""

# import modules
import json
import requests
import pandas as pd
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df




In [5]:
# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
eth_data = get_option_data('ETH')

# The data is now stored in Pandas DataFrames 'btc_data' and 'eth_data'
print('BTC data collected')
print(btc_data.head()) # Display first few rows of BTC data
print('ETH data collected')
print(eth_data.head()) # Display first few rows of ETH data

Date and time: 27/01/2024 20:48:31 , format: dd/mm/yyyy hh:mm:ss


100%|██████████| 684/684 [04:02<00:00,  2.89it/s]c:\Users\Ethan Vaz Falcao\anaconda3\lib\site-packages\pandas\core\internals\concat.py:382: RuntimeWarning: coroutine 'call_api' was never awaited
  unit = JoinUnit(blk, shape)
100%|██████████| 606/606 [03:39<00:00,  2.76it/s]

BTC data collected
   best_bid_amount  best_ask_amount  bid_iv  ask_iv underlying_index  \
0              0.0             10.0     0.0  302.28  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  213.97  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  270.94  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  195.70  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  263.13  SYN.BTC-28JAN24   

   underlying_price  mark_iv  interest_rate  best_bid_price  best_ask_price  \
0        42117.9379   104.62            0.0             0.0          0.1345   
0        42119.0537   104.62            0.0             0.0          0.0001   
0        42119.0537   101.32            0.0             0.0          0.1225   
0        42119.0537   101.32            0.0             0.0          0.0001   
0        42119.5533   100.23            0.0             0.0          0.1110   

   ...  greeks.theta  greeks.vega  greeks.gamma  greeks.delta  \
0  ...  

In [6]:
btc_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,10.0,0.0,302.28,SYN.BTC-28JAN24,42117.9379,104.62,0.0,0.0,0.1345,...,-0.00038,0.00001,0.0,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,213.97,SYN.BTC-28JAN24,42119.0537,104.62,0.0,0.0,0.0001,...,-0.00003,0.00001,0.0,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,270.94,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.1225,...,-0.00197,0.00004,0.0,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,195.70,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.0001,...,-0.00015,0.00004,0.0,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,263.13,SYN.BTC-28JAN24,42119.5533,100.23,0.0,0.0,0.1110,...,-0.01554,0.00031,0.0,0.99999,0.0,0.0,None,None,None,day


In [8]:
btc_data

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,10.0,0.0,302.28,SYN.BTC-28JAN24,42117.9379,104.62,0.0,0.0,0.1345,...,-0.00038,0.00001,0.00000,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,213.97,SYN.BTC-28JAN24,42119.0537,104.62,0.0,0.0,0.0001,...,-0.00003,0.00001,0.00000,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,270.94,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.1225,...,-0.00197,0.00004,0.00000,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,195.70,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.0001,...,-0.00015,0.00004,0.00000,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,263.13,SYN.BTC-28JAN24,42119.5533,100.23,0.0,0.0,0.1110,...,-0.01554,0.00031,0.00000,0.99999,0.0,0.0,None,None,None,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,0.0,0.00,BTC-27DEC24,43991.9600,110.67,0.0,0.0,0.0000,...,-26.27974,158.74128,0.00001,-0.63149,0.0,0.0,None,None,None,month
0,0.0,0.0,0.0,0.00,BTC-27DEC24,43991.9600,110.67,0.0,0.0,0.0000,...,-25.47913,153.90291,0.00001,0.33800,0.0,0.0,None,None,None,month
0,0.0,0.0,0.0,0.00,BTC-27DEC24,43992.0900,110.67,0.0,0.0,0.0000,...,-25.47924,153.90354,0.00001,-0.66200,0.0,0.0,None,None,None,month
0,0.0,7.9,0.0,84.75,BTC-27DEC24,43992.0900,110.67,0.0,0.0,0.0555,...,-24.61664,148.69312,0.00001,0.31083,0.0,0.0,None,None,None,month


In [7]:
eth_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.8832,222.73,0.0,0.0,0.0,...,-2.21171,0.01986,0.00025,0.98721,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-0.60758,0.01984,0.00025,-0.01278,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-5.16421,0.04637,0.00058,0.96511,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-1.88950,0.04637,0.00058,-0.03489,45.27,10.0,0.0,0.002,0.002,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-9.97799,0.08960,0.00111,0.91982,0.00,0.0,None,None,None,day
